In [1]:
#importing the packages
import pandas as pd
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import pandas_ta as ta
import http.client, urllib.parse
import json
from datetime import datetime
import time

In [5]:
#function to create the required data for each stock
def create_data(ticker,start_date,end_date):
    df = web.DataReader(name=ticker, data_source='yahoo', start=start_date, end=end_date)
    df['H-L'] = df['High'] - df['Low']
    df['O-C'] = df['Open'] - df['Close']
    df['5MA'] = df['Adj Close'].rolling(window=5).mean()
    df['10MA'] = df['Adj Close'].rolling(window=10).mean()
    df['20MA'] = df['Adj Close'].rolling(window=20).mean()
    df['7SD'] = df['Adj Close'].rolling(window=7).std()
    df["EMA8"] = df['Adj Close'].ewm(span=8).mean()
    df["EMA21"] = df['Adj Close'].ewm(span=21).mean()
    df['EMA34'] = df['Adj Close'].ewm(span=34).mean()
    df['EMA55'] = df['Adj Close'].ewm(span=55).mean()
    df.ta.rsi(close='Close', length=14, append=True)
    df['Close'] = df['Close'].shift(-1)
    df = df.reset_index()
    df.dropna(inplace=True)
    df['Sentiment'] = get_sentiment_data(ticker,df)
    #save the data in a csv file
    df.to_csv(r'../data/test_{name}.csv'.format(name=ticker))

In [8]:
#function to access the api endpoint and get the data
def get_sentiment_data(ticker_symbol,df):
    dates = df['Date']
    dates = dates.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d"))
    sentiment_values = []

    for m in dates:        
        #the request is put in a try catch block to stop the loop from breaking
        try:
            #initiate the connection to the 3rd party api service
            conn = http.client.HTTPSConnection('api.marketaux.com') 
            params = urllib.parse.urlencode({
                'api_token': 'fkkywOjEYioULrZrV9pt21k6pTtRPW5C17FeWNkE',
                'symbols': ticker_symbol,
                'published_on':m
                })

            conn.request('GET', '/v1/entity/stats/aggregation?{}'.format(params))
            res = conn.getresponse()
            data = res.read()
            parsed = json.loads(data)
            sentiment_values.append(parsed['data'][0]['sentiment_avg'])
            print(m)
        except Exception as e:
            print('The error occured is ' + e.__class__)       
        time.sleep(0.5)
        
    return sentiment_values

In [11]:
#obtain and store the stock related data
stock_tickers = ['AAPL','TSLA','GOOG','MSFT']

start_date = "2022-02-01"
end_date = "2022-04-15"

for i in stock_tickers:
    create_data(i,start_date,end_date)
    print(i + "is done")
    time.sleep(2)

2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-11
2022-04-12
2022-04-13
AAPLis done
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-11
2022-04-12
2022-04-13
TSLAis done
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-28
2022-03-29
2022-03-